# Import libraries

In [11]:
from functions.vectorization import *
from functions.labelization import *
from functions.vocabulary import *
from functions.model import *
from functions.predict import *
from functions.tools import count_words
import pandas as pd
import numpy as np

# Import data

In [12]:
path_test = '../data/test/TEST_enero_agosto_consejo_social.json'
df_test = pd.read_json(path_test)
# Set to upper the column names
df_test.columns = [x.upper() for x in df_test.columns]
# df_test['ID_PUESTO_ESCO_ULL'] = df_test['ID_PUESTO_ESCO_ULL'].apply(lambda x: str(int(x)))
df_test['NUM_WORDS'] = count_words(df_test)
df_test.head()

,ID_OFERTA,PALABRAS_EMPLEO_TEXTO,CATEGORIA,SUBCATEGORIA,NUM_WORDS
0,5ebde36ac743a8b9509e020bb9648f,CONTABILIDAD GESTION PLANIFICACIONES PRODUCCIO...,Administración de empresas,Finanzas y contabilidad,22
1,5403f26f8a4e7a882b0416977c2f6c,PREPARADOR LABORAL LUCRO CULTURAS IGUALDAD PER...,Educación y formación,Pedagogía,18
2,72d3e16e974e1e9948e9a66a3f1553,EMPRESAS LOGISTICO ALIMENTACION DEPENDENCIA CO...,"Compras, logística y almacén",Distribución y logística,52
3,7a8290f16f4bbdb39a498c15c48861,RECURSOS ELABORACION CONTRATOS NOMINA SEGUROS ...,Recursos humanos,Relaciones laborales,10
4,0fef6adf3d4ed88360462a0a60483e,GESTION HOSTELERIA RESTAURANTE FRANQUICIAS TEC...,Turismo y restauración,Hostelería,23


# Vectorize test offers and id to label

In [13]:
path_vocabulary = '../data/train/vocabulary.json'
vocabulary = load_vocabulary(path_vocabulary)
test_vectors = np.vstack(create_vectorize_dataframe(df_test, vocabulary))
path_ids_labels = '../data/train/ids_labels.json'
dict_label_ids = load_dict_label_ids(path_ids_labels)
# real_ids = list(df_test['ID_PUESTO_ESCO_ULL'])
# test_labels = cast_id_to_labels(real_ids, dict_label_ids)

# Load model

In [14]:
path_model= '../model/my_model_V15.h5'
model = load_model(path_model)

# Get accuracy and predictions

In [15]:
# Get the accuracy for this TEST
# accuracy = get_accuracy(model, test_vectors, dict_label_ids, real_ids)
# Print the value of accuracy in percentage with 2 decimals
# print('Accuracy: {:.2%}'.format(accuracy))
# Get predictions for this TEST
predictions_id, probability = get_predictions(model, test_vectors, dict_label_ids)

2023-09-05 12:37:37.976744: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 740138880 exceeds 10% of free system memory.


1268/1268 [==============================] - 3s 2ms/step


# Create report

In [16]:
df_report = pd.DataFrame({'ID_OFERTA': df_test['ID_OFERTA'],
                            'PREDICTED_ID': predictions_id,
                            'PROBABILITY': probability,
                            'NUM_OF_WORDS': df_test['NUM_WORDS']})
df_report.head()
df_report.to_csv('../reports/test_estudio_consejo_social_enero_agosto.csv', index=False)